In [1]:
import os
import copy
import warnings

import torch
import random
import pandas as pd
import numpy  as np
import torch.nn as nn
from src.processing import *
from torchvision import models
from torch.utils.data import DataLoader
import src.model_fitting as model_fitting

warnings.filterwarnings("ignore")

SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

!nvidia-smi

C:\Users\ozher\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Thu Nov  3 18:16:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 462.62       Driver Version: 462.62       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 305... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   43C    P0    11W /  N/A |    106MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
images_amount = check_images_amount()
print(images_amount)

annotations = pd.read_csv("annotation_file.csv")
train, valid = train_val_split(annotations)

if(not valid.shape[0] + train.shape[0] == images_amount["train"]):
    raise ValueError("Sum of len(valid) and len(train) is not equal to total amount of train images")

{'test': 1436, 'train': 3802}


In [3]:
NEW_SIZE, BATCH_SIZE = 224, 64
train_dataset = CarDataset(annotation_file=train, img_dir="train", new_size=NEW_SIZE, mode="train")
valid_dataset = CarDataset(annotation_file=train, img_dir="train", new_size=NEW_SIZE, mode="valid")

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [4]:
model_resnet18 = models.resnet18(pretrained=True)
model_resnet18.fc = nn.Linear(512, 11)
model_resnet18 = model_resnet18.cuda()

torch.cuda.empty_cache()

In [5]:
fitting_info = model_fitting.train(train_dataloader, valid_dataloader, model=model_resnet18, epochs=10)

torch.save(copy.deepcopy(model_resnet18.state_dict()), 'model_resnet18_weights.pth')
torch.cuda.empty_cache()

epoch:  10%|████████████████▋                                                                                                                                                      | 1/10 [00:49<07:22, 49.14s/it]

[0]: tr_loss: 0.53157 val_loss: 0.12522            tr_f1: 0.80839 val_f1: 0.9564


epoch:  20%|█████████████████████████████████▍                                                                                                                                     | 2/10 [01:53<07:42, 57.87s/it]

[1]: tr_loss: 0.38129 val_loss: 0.47789            tr_f1: 0.8541 val_f1: 0.84224


epoch:  30%|██████████████████████████████████████████████████                                                                                                                     | 3/10 [02:56<07:03, 60.52s/it]

...

[7]: tr_loss: 0.00704 val_loss: 0.00563            tr_f1: 0.99769 val_f1: 0.99868


epoch:  90%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                | 9/10 [09:19<01:03, 63.45s/it]

[8]: tr_loss: 0.00702 val_loss: 0.00402            tr_f1: 0.99741 val_f1: 0.99921


epoch: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [10:22<00:00, 62.29s/it]


[9]: tr_loss: 0.00415 val_loss: 0.0034            tr_f1: 0.99842 val_f1: 0.99921


In [6]:
inverse_target_mapping = {path_idx: path for path_idx, path in enumerate(os.listdir("train"))} 

annot_test = pd.read_csv("annotation_file_test.csv").sort_values(by="id")

test_dataset = CarDataset(annotation_file=annot_test, img_dir="public_test", new_size=NEW_SIZE, mode="test")
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

preds = model_fitting.predict(model_resnet18, test_dataloader, inverse_target_mapping=inverse_target_mapping)
if((len(preds) == images_amount["test"]) and (len(np.unique(preds)) == 11)):
    make_submit("submit", preds)

=====SUBMITION DONE=====


----